# Project 1b

#### Importing libraries

In [1]:
import numpy as np
import pandas as pd

# Add any additional imports here (however, the task is solvable without using any additional imports)
from sklearn import linear_model, model_selection
from sklearn.model_selection import KFold

 #### Loading data

In [2]:
data = pd.read_csv("train.csv")
y = data["y"].to_numpy()
data = data.drop(columns=["Id", "y"])
# print a few data samples
print(data.head())
X = data.to_numpy()

     x1    x2    x3    x4    x5
0  0.02  0.05 -0.09 -0.43 -0.08
1 -0.13  0.11 -0.08 -0.29 -0.03
2  0.08  0.06 -0.07 -0.41 -0.03
3  0.02 -0.12  0.01 -0.43 -0.02
4 -0.14 -0.12 -0.08 -0.02 -0.08


#### Transforming data

In [3]:
"""
Transform the 5 input features of matrix X (x_i denoting the i-th component of X) 
into 21 new features phi(X) in the following manner:
5 linear features: phi_1(X) = x_1, phi_2(X) = x_2, phi_3(X) = x_3, phi_4(X) = x_4, phi_5(X) = x_5
5 quadratic features: phi_6(X) = x_1^2, phi_7(X) = x_2^2, phi_8(X) = x_3^2, phi_9(X) = x_4^2, phi_10(X) = x_5^2
5 exponential features: phi_11(X) = exp(x_1), phi_12(X) = exp(x_2), phi_13(X) = exp(x_3), phi_14(X) = exp(x_4), phi_15(X) = exp(x_5)
5 cosine features: phi_16(X) = cos(x_1), phi_17(X) = cos(x_2), phi_18(X) = cos(x_3), phi_19(X) = cos(x_4), phi_20(X) = cos(x_5)
1 constant feature: phi_21(X)=1

Parameters
----------
X: matrix of floats, dim = (700,5), inputs with 5 features

Compute
----------
X_transformed: array of floats: dim = (700,21), transformed input with 21 features
"""
X_transformed = np.zeros((700, 21))

# TODO: Enter your code here
X_transformed = np.hstack((X, np.square(X), np.exp(X), np.cos(X), np.ones((700, 1))))
#####

assert X_transformed.shape == (700, 21)

#### Performing model selection

In [4]:
# lasso regularization, k-fold CV to choose lambda
lambdas = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08] # slow convergence for higher lambdas
n_folds = 5
RMSE_mat = np.zeros((n_folds, len(lambdas)))
kf = KFold(n_splits=n_folds)
j = 0
for lam in lambdas:
    i = 0
    for train, test in kf.split(X):
        x_train, x_test, y_train, y_test = X_transformed[train], X_transformed[test], y[train], y[test]
        reg = linear_model.Lasso(alpha=lam, fit_intercept=False, random_state=42)
        reg.fit(x_train, y_train)
        w = reg.coef_
        n = len(y_test)
        RMSE_mat[i, j] = np.sqrt((1/n) * (np.linalg.norm(y_test - (x_test@w))**2))
        i += 1
    j += 1
avg_RMSE = np.mean(RMSE_mat, axis=0)
print(avg_RMSE)

[1.9600833  1.95872163 1.95812307 1.9581467  1.95840483 1.95872958
 1.95916479 1.95968563]


In [5]:
# lasso regularization, lambda = 0.03 (lowest cross-validation error)
reg = linear_model.Lasso(alpha=0.03, fit_intercept=False, random_state=42)
reg.fit(X_transformed, y)
w = reg.coef_
print("Coefficients: \n", w)

Coefficients: 
 [ 0.         -0.         -0.          0.          0.         -0.
  0.          0.         -0.          0.         -0.10314805 -1.71780757
 -2.41998714 -0.         -0.00797411 -0.72869001 -1.52968372 -0.
 -0.         -0.01020332 -0.        ]


In [6]:
# ridge regularization, k-fold CV to choose lambda
lambdas = [0.1, 0.5, 1, 5, 10, 20, 50, 100]
n_folds = 5
RMSE_mat = np.zeros((n_folds, len(lambdas)))
kf = KFold(n_splits=n_folds)
j = 0
for lam in lambdas:
    i = 0
    for train, test in kf.split(X):
        x_train, x_test, y_train, y_test = X_transformed[train], X_transformed[test], y[train], y[test]
        reg = linear_model.Ridge(alpha=lam, fit_intercept=False, random_state=42)
        reg.fit(x_train, y_train)
        w = reg.coef_
        n = len(y_test)
        RMSE_mat[i, j] = np.sqrt((1/n) * (np.linalg.norm(y_test - (x_test@w))**2))
        i += 1
    j += 1
avg_RMSE = np.mean(RMSE_mat, axis=0)
print(avg_RMSE)

[1.96878705 1.96469275 1.96317828 1.96047518 1.9601682  1.96025772
 1.96083767 1.96292632]


#### Fitting data

In [7]:
"""
Use the transformed data points X_transformed and fit the linear regression on this 
transformed data. Finally, compute the weights of the fitted linear regression. 

Parameters
----------
X_transformed: array of floats: dim = (700,21), transformed input with 21 features
y: array of floats, dim = (700,), input labels)

Compute
----------
w: array of floats: dim = (21,), optimal parameters of linear regression
"""
w = np.zeros((21,))

# TODO: Enter your code here
# ridge regularization, lambda = 10 (lowest cross-validation error)
reg = linear_model.Ridge(alpha=10, fit_intercept=False, random_state=42)
reg.fit(X_transformed, y)
w = reg.coef_
print("Coefficients: \n", w)
#####

assert w.shape == (21,)

Coefficients: 
 [ 0.13452424 -0.31706357 -0.46497623  0.22050574  0.08450269 -0.16125329
  0.08779552  0.08831434 -0.11734824  0.03443236 -0.51061588 -0.83693986
 -0.98700956 -0.39820775 -0.46428312 -0.48169923 -0.60546589 -0.60568324
 -0.50433045 -0.5782814  -0.56181812]


In [ ]:
# Save results in the required format
np.savetxt("./results.csv", w, fmt="%.12f")